<a href="https://colab.research.google.com/github/j-river1/Sensores_Caudal/blob/master/Sensores_Caudal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Visualización y tratamiento de los datos de los sensores de caudal**



##1. Preprocesamiento de los datos

>Requisitos de formato y organzación de los archivos.

El tipo de archivos para este procesamiento debe ser de tipo JSON ya que es el formato original de datos que proviene de los sensores. Si los archivos están guardados en otro tipo de extensión como TXT se debe cambiar a JSON, en estas paginas [[1]](https://www.isumsoft.com/windows-10/change-file-extension-for-one-or-multiple-files.html#:~:text=Another%20way%20to%20open%20the,name), [[2]](https://www.isumsoft.com/windows-10/rename-multiple-files-at-once-in-windows-10.html)
 encontraran algunas herramientas para convertir multiples archivos simultáneamente. Además, todo los archivos deben estar guardado en una sola carpeta.




##2. Lectura de datos

> Conexión a los datos

Para la lectura de datos se debe copiar la carpeta en google drive. Una vez guardados, se realiza la conexión con este notebook. Es importante mencionar que este notebook esta desarrollado en Coolab por lo tanto para la conexión se crea por medio de la opcion Unmount drive, se ingresa en la parte izquierda (Files -> Amount Dive). O también puede crear la conexión mediante el siguiente comando, una vez autenticados se debe copiar la contraseña en el link habilitado. Para mayor información puede consultar en [[3]](https://www.marktechpost.com/2019/06/07/how-to-connect-google-colab-with-google-drive/), [[4]](https://www.isumsoft.com/windows-10/rename-multiple-files-at-once-in-windows-10.html).


In [7]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Auntetificación para crear la conexión.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


Ya creada la conexión, se debe seleccionar la carpeta de los datos, generalmente está ubicada en la carpeta *content/drive*, estos archivos están ubicados en el lado izquierdo de Coolab. A continuación se lee el archivo con los datos. 


### 1.2 Unión de los archivos JSON

Una vez se tiene los archivos JSON se procede a unirlos en un solo archivo.  Generalmente los datos enviados desde
los sensores están guardados por dia  y están organizados por meses, por ejemplo, en la siguiente figura se muestra los meses de enero a septiembre (a) y los nueve primeros dias del mes de septiembre.

<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="https://github.com/j-river1/Sensores_Caudal/blob/master/OrganizacionFolderes_2.png?raw=true", align="left">



> Lectura de los datos

Para la lectura de los datos se debe guardar los datos en goolge Drive, como se muestra en la siguiente figura. Se debe ingresar por files y activar la opción de unmount Drive (1), una vez activada la opción aparece la carpeta drive (2).

<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="https://github.com/j-river1/Sensores_Caudal/blob/master/ConexionDrive.png?raw=true", align="left">



In [5]:
from os import listdir
from os.path import isfile, join

mypath = '/content/drive/My Drive/Datos sensores caudal/2019/01'

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

onlyfiles 


['03.json',
 '18.json',
 '17.json',
 '20.json',
 '23.json',
 '04.json',
 '25.json',
 '10.json',
 '12.json',
 '02.json',
 '30.json',
 '07.json',
 '19.json',
 '09.json',
 '16.json',
 '24.json',
 '31.json',
 '22.json',
 '11.json',
 '21.json',
 '05.json',
 '29.json',
 '15.json',
 '13.json',
 '26.json',
 '06.json',
 '28.json',
 '14.json',
 '01.json',
 '08.json',
 '27.json']

#Referencias 
[1]. https://www.isumsoft.com/windows-10/change-file-extension-for-one-or-multiple-files.html#:~:text=Another%20way%20to%20open%20the,name)%2C%20and%20press%20Enter.
[2] https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92